In [1]:
import pandas as pd
import numpy as np

## Get the years_active feature

In [8]:
companies = pd.read_csv('.././data/feng/companies.csv').convert_dtypes()
# Generate the years active feature
companies['years_active'] = 2022 - companies.founded
# send it to EDA
companies.to_csv('.././data/EDA/companies.csv', index=False)

## Gerate features from the financial statemens to send the EDA

First Feature trim

In [37]:
pd.options.display.max_columns = 80

In [60]:
to_drop = [
    'reportedCurrency',
    'cik',
    'acceptedDate',
    'period',
    'deferredIncomeTax',
    'stockBasedCompensation',
    'changeInWorkingCapital',
    'accountsReceivables',
    'inventory',
    'accountsPayables',
    'otherWorkingCapital',
    'otherNonCashItems',
    'acquisitionsNet',
    'purchasesOfInvestments',
    'salesMaturitiesOfInvestments',
    'netCashProvidedByOperatingActivities',
    'investmentsInPropertyPlantAndEquipment',
    'otherInvestingActivites',
    'netCashUsedForInvestingActivites',
    'debtRepayment',
    'commonStockIssued',
    'commonStockRepurchased',
    'otherFinancingActivites',
    'otherFinancingActivites',
    'netCashUsedProvidedByFinancingActivities',
    'dividendsPaid',
    'netChangeInCash',
    'cashAtBeginningOfPeriod',
    'cashAtEndOfPeriod',
    'effectOfForexChangesOnCash',
    'link',
    'finalLink',
    'grossProfit',
    'operatingExpenses',
    'operatingExpenses',
    'costAndExpenses',
    'depreciationAndAmortization',
    'ebitda',
    'operatingIncome',
    'totalOtherIncomeExpensesNet',
    'incomeBeforeTax',
    'incomeTaxExpense',
    'netIncome',
    'eps',
    'epsdiluted',
    'weightedAverageShsOut',
    'weightedAverageShsOutDil',
    'totalCurrentAssets',
    'propertyPlantEquipmentNet',
    'goodwill',
    'intangibleAssets',
    'goodwillAndIntangibleAssets',
    'longTermInvestments',
    'taxAssets',
    'otherNonCurrentAssets',
    'totalNonCurrentAssets',
    'otherAssets',
    'otherNonCurrentAssets',
    'otherNonCurrentAssets',
    'taxPayables',
    'deferredRevenue',
    'otherCurrentLiabilities',
    'deferredRevenueNonCurrent',
    'deferredTaxLiabilitiesNonCurrent',
    'otherNonCurrentLiabilities',
    'otherLiabilities',
    'capitalLeaseObligations',
    'preferredStock',
    'commonStock',
    'accumulatedOtherComprehensiveIncomeLoss',
    'othertotalStockholdersEquity',
    'totalStockholdersEquity',
    'totalLiabilitiesAndStockholdersEquity',
    'minorityInterest',
    'totalLiabilitiesAndStockholdersEquity',
    ]

In [61]:
statements = pd.read_csv('.././data/feng/statements_y.csv')
statements = statements.drop(columns=to_drop)

Adjust for inflation

In [68]:
statements['date'] = pd.to_datetime(statements['date'])
statements['month'] = statements['date'].dt.month

In [69]:
inflation = pd.read_csv('.././data/feng/inflation.csv').convert_dtypes()
statements = statements.merge(inflation.loc[:,['year','month', 'multiplier']], how='left', left_on=['calendarYear', 'month'], right_on = ['year', 'month'])

In [71]:
statements.to_csv('.././data/EDA/statements.csv', index=False)

IDEAS
- Net income (income-statement)
- EBIT/MarketCap ()
- Enterprise Value: Íncludes debt in market cap
- Free cash flow to Mcap
- Revenue, NetIncome, growth_yoy for each quarter
- ROIC: How much extra value a company creates for unit invested
- WACC: Cost of capital
- Cash/debt
- mcap to revenue
- Interest coverage
- NRE

### Get Inflation coefficient from corePCE

In [ ]:
# read inflation csv
inflation = pd.read_csv('.././data/fred/corePCE.csv').convert_dtypes()
# get last date
last_date = inflation.date.idxmax()
# get the coefficient of that date
today_inflation = inflation.loc[last_date, 'corePCE']
# calculate multiplier
inflation['multiplier'] = today_inflation/inflation.corePCE
# export csv
inflation.to_csv('.././data/feng/inflation.csv', index=False)

### Get Real Market Caps

In [2]:
mcaps = pd.read_csv('.././data/feng/mcaps.csv').convert_dtypes()
inflation = pd.read_csv('.././data/feng/inflation.csv').convert_dtypes()

In [4]:
mcaps['date'] = pd.to_datetime(mcaps['date'])
mcaps['year'] = mcaps['date'].dt.year
mcaps['month'] = mcaps['date'].dt.month

In [5]:
joincols = ['year', 'month']
rightcols = ['multiplier']
mcaps = mcaps.merge(
    inflation.loc[:,joincols + rightcols],
    how='left',
    on = joincols
)

In [6]:
mcaps['real_mcap'] = mcaps.marketCap * mcaps.multiplier

smoooth the market caps with the 7-day moving average

In [7]:
# sort by symbol and date
mcaps = mcaps.sort_values(by=['symbol', 'date'])
mcaps = mcaps.reset_index(drop = True)
# get the 10-day sma10
mcaps['sma10'] = mcaps.groupby('symbol')['real_mcap'].rolling(10).mean().reset_index(drop=True)
mcaps['sma30'] = mcaps.groupby('symbol')['real_mcap'].rolling(30).mean().reset_index(drop=True)
# fill nans with real values
mcaps['sma10'] = mcaps['sma10'].fillna(mcaps['real_mcap'])
mcaps['sma30'] = mcaps['sma30'].fillna(mcaps['real_mcap'])


Send to EDA

In [8]:
mcaps.to_csv('.././data/EDA/mcaps.csv', index=False)

### Get realGDP from GDP

In [4]:
gdp = pd.read_csv('.././data/fred/GDP.csv').convert_dtypes()
inflation = pd.read_csv('.././data/feng/inflation.csv').convert_dtypes()

In [6]:
joincols = ['year', 'month']
rightcols = ['multiplier']
gdp = gdp.merge(
    inflation.loc[:,joincols + rightcols],
    how='left',
    on = joincols
)

In [8]:
gdp['realGDP'] = gdp.GDP * gdp.multiplier

In [10]:
gdp.to_csv('.././data/EDA/GDP.csv', index=False)